In [128]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import bisplrep, bisplev
from scipy import integrate

In [184]:
def create_knots(k,n,min=0,max=10):
    # Create open-uniform knot vector for a B-spline of degree k and with n+1 control points between the min and max values.  
    return np.concatenate(
        [(min)*np.zeros(k),
         np.linspace(0,max,n),
         (max)*np.ones(k)]
        )    

def fit_Bspline(x,y,z,k,tx,ty):
    #Fit B-spline on the given dataset of x,y,z and return its tck form and the depth-error.
    tck,depth_loss,_,_=bisplrep(x, y, z, kx=k, ky=k, task=-1, s=0, tx=tx, ty=ty, full_output=1)
    return tck, depth_loss

def curvature_loss(tck, min_x=0, max_x=10, min_y=0, max_y=10):
    #Integrate the second derivative of the B-spline, parameterized by tck, w.r.t. x and y over the domain of [min_x,max_x] x [min_y,max_y]
    #Computing a double integral is rather computationally expensive, therefore a trick is used to only use singular integrals. Alternatively, the Leibniz rule could be used.
    bs_x = lambda y: bisplev(max_x,y,tck,1,0)-bisplev(min_x,y,tck,1,0)
    bs_y = lambda x: bisplev(x,max_y,tck,0,1)-bisplev(x,min_y,tck,0,1)    

    curv_loss_x,_ = integrate.quad(bs_x ,min_y,max_y)
    curv_loss_y,_ = integrate.quad(bs_y ,min_x,max_x)

    curv_loss = curv_loss_x + curv_loss_y
    return curv_loss
    

In [187]:
#This is just an example.

#Generate random data points
np.random.seed(42)  # For reproducibility
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
z = np.random.random((100, 100))

# Create a meshgrid from x and y
X, Y = np.meshgrid(x, y)

# Flatten the meshgrid arrays and z array to pass to bisplrep
x_flat = X.flatten()
y_flat = Y.flatten()
z_flat = z.flatten()

# Obtain the border values of the domain. 
max_x=np.max(x_flat)
min_x=np.min(x_flat)
max_y=np.max(y_flat)
min_y=np.min(y_flat)

#Define the parameters
k =3 #Degree (both x and y direction)
n = 10 # The number of control points (n+1) 

# Create open 
tx=create_knots(k,n, min_x, max_x)
ty=create_knots(k,n, min_y, max_y)

#Create a B-Spline representation using previous determined knots on the data and store the depthmap losserror. 
tck,dm_loss=fit_Bspline(x_flat, y_flat, z_flat, k=k, tx=tx, ty=ty)

#Compute the curvature loss
curv_loss=curvature_loss(tck, min_x, max_x, min_y, max_y)

#print the losses
print("Depthmap loss:", dm_loss)
print("Curvature loss:", curv_loss)


Depthmap loss: 816.1902474992237
Curvature loss: -2.2293483560146647
